In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, accuracy_score
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

def extensive_grid_search(df, target_cols):
    """
    Perform extensive grid search for both feature selection and classification.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe
    target_cols : list
        List of target columns to exclude from features
    """
    # Prepare data
    le = LabelEncoder()
    df_encoded = df.copy()
    for column in df_encoded.select_dtypes(include=['object']).columns:
        df_encoded[column] = le.fit_transform(df_encoded[column].astype(str))

    X = df_encoded.drop(columns=target_cols, errors='ignore')
    y = df_encoded['target'].astype(int)

    # Create pipeline
    pipeline = ImbPipeline([
        ('sampling', SMOTE(random_state=42)),
        ('feature_selection', SelectFromModel(GradientBoostingClassifier())),
        ('classifier', GradientBoostingClassifier())
    ])

    # Define parameter grid
    param_grid = {
        # Feature Selection parameters
        'feature_selection__threshold': ['mean', '0.5*mean', '1.5*mean'],
        'feature_selection__max_features': [5, 7, 9, 11, 13],
        'feature_selection__estimator__n_estimators': [50, 100],
        
        # Classifier parameters
        'classifier__n_estimators': [100, 150, 200],
        'classifier__learning_rate': [0.05, 0.1, 0.15],
        'classifier__max_depth': [3, 4, 5, 6],
        'classifier__min_samples_split': [5, 8, 10],
        'classifier__min_samples_leaf': [2, 3, 4],
        'classifier__subsample': [0.8, 0.9, 1.0],
        'classifier__max_features': ['sqrt', 'log2', None]
    }

    # Define scoring metrics
    scoring = {
        'roc_auc': 'roc_auc',
        'f1': 'f1',
        'accuracy': 'accuracy'
    }

    # Create GridSearchCV
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=scoring,
        refit='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=2,
        return_train_score=True
    )

    # Fit GridSearchCV
    print("Starting Grid Search...")
    start_time = time()
    grid_search.fit(X, y)
    print(f"Grid Search completed in {(time() - start_time)/60:.2f} minutes")

    # Get results into DataFrame
    results = pd.DataFrame(grid_search.cv_results_)
    
    # Get best parameters and scores
    print("\nBest parameters found:")
    print(grid_search.best_params_)
    print("\nBest scores:")
    print(f"ROC AUC: {grid_search.best_score_:.4f}")

    # Analyze results
    analyze_grid_search_results(results)
    
    return grid_search, results

def analyze_grid_search_results(results):
    """
    Analyze and visualize grid search results.
    """
    # Create figure for multiple plots
    fig, axes = plt.subplots(2, 2, figsize=(20, 15))
    
    # 1. Parameter importance plot
    param_scores = []
    for param in results.params.iloc[0].keys():
        scores = []
        values = results[f'param_{param}'].unique()
        for value in values:
            mean_score = results[
                results[f'param_{param}'] == value
            ]['mean_test_roc_auc'].mean()
            scores.append({'param': param, 'value': str(value), 'score': mean_score})
        param_scores.extend(scores)
    
    param_importance_df = pd.DataFrame(param_scores)
    
    # Plot parameter importance
    sns.boxplot(
        data=param_importance_df,
        x='param',
        y='score',
        ax=axes[0,0]
    )
    axes[0,0].set_title('Parameter Importance')
    axes[0,0].set_xticklabels(axes[0,0].get_xticklabels(), rotation=45)
    
    # 2. Score distributions
    scores_df = results[['mean_test_roc_auc', 'mean_test_f1', 'mean_test_accuracy']]
    scores_df.boxplot(ax=axes[0,1])
    axes[0,1].set_title('Score Distributions')
    
    # 3. Top 10 combinations
    top_10_idx = results['mean_test_roc_auc'].nlargest(10).index
    top_10_scores = results.iloc[top_10_idx][
        ['mean_test_roc_auc', 'mean_test_f1', 'mean_test_accuracy']
    ]
    top_10_scores.plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Top 10 Combinations')
    axes[1,0].set_xticklabels(range(1, 11))
    
    # 4. Learning curves for best model
    best_idx = results['mean_test_roc_auc'].idxmax()
    train_scores = results.iloc[best_idx][
        [col for col in results.columns if 'split' in col and 'train' in col]
    ]
    test_scores = results.iloc[best_idx][
        [col for col in results.columns if 'split' in col and 'test' in col]
    ]
    
    axes[1,1].plot(train_scores, label='Train')
    axes[1,1].plot(test_scores, label='Test')
    axes[1,1].set_title('Learning Curves (Best Model)')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print("\nTop 5 Parameter Combinations:")
    top_5_idx = results['mean_test_roc_auc'].nlargest(5).index
    for idx in top_5_idx:
        print("\nParameters:")
        for param, value in results.params.iloc[idx].items():
            print(f"{param}: {value}")
        print(f"ROC AUC: {results.iloc[idx]['mean_test_roc_auc']:.4f}")
        print(f"F1 Score: {results.iloc[idx]['mean_test_f1']:.4f}")
        print("="*50)

def train_best_model(grid_search, X, y):
    """
    Train model with best parameters found.
    """
    # Get best parameters
    best_params = grid_search.best_params_
    
    # Create pipeline with best parameters
    best_pipeline = ImbPipeline([
        ('sampling', SMOTE(random_state=42)),
        ('feature_selection', SelectFromModel(
            GradientBoostingClassifier(**{k.replace('feature_selection__estimator__', ''): v 
                                        for k, v in best_params.items() 
                                        if 'feature_selection__estimator__' in k}),
            **{k.replace('feature_selection__', ''): v 
               for k, v in best_params.items() 
               if 'feature_selection__' in k and 'estimator__' not in k}
        )),
        ('classifier', GradientBoostingClassifier(**{k.replace('classifier__', ''): v 
                                                   for k, v in best_params.items() 
                                                   if 'classifier__' in k}))
    ])
    
    # Fit pipeline
    best_pipeline.fit(X, y)
    
    return best_pipeline

# Execute grid search
target_columns = ["target", "T_score_fn", "T_score_tf", "T_score_sp",
                 "DXXSPN_DXXOSBMD", "DXXFEM_DXXNKBMD", "DXXFEM_DXXOFBMD"]

print("Starting grid search process...")
grid_search, results = extensive_grid_search(df_final, target_columns)

# Train best model
X = df_final.drop(columns=target_columns, errors='ignore')
y = df_final['target'].astype(int)
best_model = train_best_model(grid_search, X, y)

print("\nBest model has been trained and is ready for use!")